In [ ]:
########################################
# 4단계: 모델 수정 및 미세 조정 설정 (Fine-tuning Setup)
########################################

In [ ]:
# 모델을 읽어오고
model_name = "gpt2-small-124M.pth"
model = load_gpt2_model(model_name, BASE_CONFIG)

In [ ]:
# 1. 모든 파라미터를 고정(Freeze)하여 학습되지 않게 설정
for param in model.parameters():
    param.requires_grad = False
# 특정 파라미터(예: 모델의 가중치)에 대해 영구적으로 gradient 추적을 끄는 설정
# 이 파라미터는 앞으로 학습 과정에서 업데이트되지 않는다. 
# 주로 모델 일부를 freeze할 때 사용
# requires_grad=True 로 바꾸면 학습 가능한 상태가 됨.

In [ ]:
# 2. 출력층(Head) 교체: 기존 50257개 단어 예측 -> 2개 클래스(스팸/햄) 예측
# 이 새로운 층은 기본적으로 requires_grad=True 상태임
num_classes = 2
# region [출력층(Head) 교체]
model.out_head = torch.nn.Linear(in_features=BASE_CONFIG["emb_dim"], out_features=num_classes)
# endregion

In [ ]:
# 3. 추가적으로 마지막 트랜스포머 블록과 정규화 층을 학습 가능하도록 잠금 해제 (Unfreeze)
# -> 이렇게 하면 출력층과 모델의 마지막 부분만 데이터에 맞게 조정됨 (효율적 학습)
# region [마지막 트랜스포머 블록과 정규화 층 잠금 해제]
for param in model.trf_blocks[-1].parameters():
    param.requires_grad = True

for param in model.final_norm.parameters():
    param.requires_grad = True

In [ ]:
########################################
# 5단계: 모델 미세 조정 (Fine-tuning) 실행
########################################

# 옵티마이저 설정 (학습 가능한 파라미터만 전달됨)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5, weight_decay=0.1)

In [ ]:
num_epochs = 5
# 학습 함수 호출
train_losses, val_losses, train_accs, val_accs, examples_seen = train_classifier_simple(
    model, train_loader, val_loader, optimizer, device,
    num_epochs=num_epochs, eval_freq=50, eval_iter=5,
)